## 动量策略（三）

In [ ]:
#==================================================================================================
# 动量策略（三）
#导包操作
from datetime import timedelta, date
import pandas as pd
#==================================================================================================
#==================================================================================================
#初始化函数，设置初始条件
def init(context):
    context.n = 50
    #设置最大持股
    run_monthly(trade,date_rule=2)
    #设置trade函数能每个月月初第二个交易日运行
    get_iwencai('未停牌,上市时间超过2年')
    #使用问财进行选股，每日执行一次，储存在context.iwencai_securities对象中
#==================================================================================================
#==================================================================================================
#设置止损止盈，handle_bar是每日（分钟/tick）运行
def handle_bar(context,bar_dict):
    #获取账户持仓信息
    holdstock = list(context.portfolio.stock_account.positions.keys()) 
    if len(holdstock) > 0:
        num = -0.1
        for stock in holdstock:
            close = history(stock,['close'],1,'1d').values
            if close/context.portfolio.positions[stock].last_price -1 <= num:
                order_target(stock,0)
                log.info('股票{}已止损'.format(stock))
#==================================================================================================
#==================================================================================================
#选股函数，这里用字典选股
def stocks_zf(context,bar_dict):
    df = {'security': [], '30zf': []}
    #创建字典，格式如上。'security'是键，[]是值。
    stocks=context.iwencai_securities
    #将问财选股结果导到stocks
    for symbol in stocks:
        df['security'].append(symbol)
        #将依次选出的个股添加到字典的键'security'的值中，这里append是添加操作。
    for i in range(len(df['security'])):
        quote = history(df['security'][i], ['quote_rate'], 30, '1d', True, fq='pre')
        #获取过去30日涨跌幅数据
        AMP30 = quote.values[:].sum()
        #将这些数据求和，直接是.sum()
        df['30zf'].append(AMP30)
        #将计算出的值添加到键中
    df = pd.DataFrame(df).sort_values(by ='30zf', ascending=False)
    #pd.bar_dictFrame(df)将字典转化成bar_dictFrame格式，.sort_values是排序，by是以XX为标准排序，ascending=False
    #是从大到小排序。
    context.sample = df['security'][:50]
    #截取排序后的DF格式的前50个股票，并输出
    return context.sample
#==================================================================================================
#==================================================================================================
def trade(context, bar_dict):
    zf_list = stocks_zf(context,bar_dict)
    #将选股函数的结果导到zf_list中，用于交易函数运行。
    stock_list = list(set(zf_list))
    if len(list(context.portfolio.stock_account.positions.keys())) > 0:
        for stock in list(list(context.portfolio.stock_account.positions.keys())):
            if stock not in stock_list:
                order_target(stock, 0)
    if len(stock_list) > 0:
        for stock in stock_list:
            if stock not in list(list(context.portfolio.stock_account.positions.keys())):
                if len(list(context.portfolio.stock_account.positions.keys())) < context.n :
                    number = context.n  - len(list(context.portfolio.stock_account.positions.keys()))
                    order_value(stock,context.portfolio.stock_account.available_cash/number)
                else: 
                    order_value(stock,context.portfolio.stock_account.available_cash)


